In [ ]:
import pandas as pd
import numpy as np
from string import digits
from tqdm.notebook import tqdm
import glob
from multiprocessing import Pool
import random
from functools import partial
import os

# huggingface lib bert
print('Loading transformers lib...')
!pip install transformers
from transformers import AutoTokenizer

Loading transformers lib...


In [ ]:
print('Loading Greek_Bert tokenizer...')
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1",use_fast=True)
# print('Loading Legal_Bert tokenizer...')
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased",use_fast=True)
# print('Loading M_Bert tokenizer...')
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased",use_fast=True)

Loading BERT tokenizer...


###RAPTARCHIS dataset fragmentation ratio calculation

In [ ]:
dataset_path = "/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/"

print("loading training set...")
train_df = pd.read_pickle(dataset_path + "train.pkl")
train_size = len(train_df)
# delete usless stuff
train_df = train_df.drop(['title', 'type', 'year', 'law_id', 'leg_uri'], axis=1)

loading training set...


In [ ]:
total_tokens = 0
total_words = 0
for _, row in tqdm(train_df.iterrows(), total=train_size):
  text = row['header'] + ' ' + row['articles']
  total_words += len(text.split()) 
  encoded = tokenizer.tokenize(
                      text,
                      add_special_tokens = False,
                      padding = False,
                      truncation = False,
                      return_attention_mask = False,
                      return_token_type_ids = False)
  total_tokens += len(encoded)

In [ ]:
print(round(total_tokens/total_words,3))

1.646


###Greek dataset fragmentation ratio calculation (greek-bert's dataset)

In [ ]:
filenames = glob.glob('/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/Greek-Bert/normalized/europal/*')
filenames += random.sample(glob.glob('/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/Greek-Bert/normalized/oscar/*'), 1)
filenames += glob.glob('/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/Greek-Bert/normalized/wiki/*')

def count(filename):
  total_tokens = 0
  total_words = 0
  block_size = 2048
  with open(filename, 'r') as f:
    def read_block():
      return f.read(block_size)
    # read file by blocks. (i dont know why total /1.8, but it almost works)
    for block in tqdm(iter(read_block, ''),
                      total=int(os.path.getsize(filename)/block_size/1.8)):
      total_words += len(block.split()) 
      encoded = tokenizer.tokenize(
                          block,
                          add_special_tokens = False,
                          padding = False,
                          truncation = False,
                          return_attention_mask = False,
                          return_token_type_ids = False)
      total_tokens += len(encoded)
  return (total_tokens, total_words)

# with Pool(processes=4) as pool:
#   results = list(tqdm(pool.imap(count, filenames), total=len(filenames)))
# print(results)
for f in tqdm(filenames):
  print(count(f))


(39595468, 35991648)



(227744659, 195004771)

